# Imports

In [1]:
import pandas as pd
import numpy as np

from mamimo.time_utils import add_time_features
from mamimo.carryover import ExponentialCarryover, GeneralGaussianCarryover
from mamimo.saturation import ExponentialSaturation, BoxCoxSaturation, AdbudgSaturation, HillSaturation

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import scipy.stats as stats

import holidays

import seaborn as sns
import matplotlib.pyplot as plt

# Load DataFrame

In [2]:

df = pd.read_csv('../../../raw_data/df.csv')
df = df.drop(columns='Unnamed: 0')
df['Day'] = pd.to_datetime(df['Day'])
df.set_index('Day', inplace=True)
df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})
df['day'] = df.index # we will use this in our column transformer
df = df.drop(columns = ['fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks', 'orders'])

# Split Data

In [3]:
n_splits = 5  # Number of splits for cross-validation
tscv = TimeSeriesSplit(n_splits=n_splits)

for train_index, test_index in tscv.split(df):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]


y_train = train_df['total_sales']
X_train = train_df.drop(columns = ['total_sales'])


y_test = test_df['total_sales']
X_test = test_df.drop(columns = ['total_sales'])

In [4]:
X_train

,facebook,google,tiktok,day
Day,,,,
2021-07-01,257.01,1.17,0.00,2021-07-01
2021-07-02,250.30,0.84,0.00,2021-07-02
2021-07-03,248.59,0.72,0.00,2021-07-03
2021-07-04,258.05,0.07,0.00,2021-07-04
2021-07-05,257.47,0.38,0.00,2021-07-05
...,...,...,...,...
2023-04-12,885.33,208.52,209.03,2023-04-12
2023-04-13,882.17,186.47,208.82,2023-04-13
2023-04-14,662.08,160.69,216.38,2023-04-14


# Create Custom Tranformers

## Time Features

### Holidays

In [5]:
class AddHolidaysTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        """Creates a new column with row value = 1 if the day is a Friday or Saturday and 0 if not."""
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        de_holiday_list = []
        for holiday in holidays.Germany(years=[2021,2022,2023]).items():
            de_holiday_list.append(holiday)
        de_holidays_df = pd.DataFrame(de_holiday_list, columns=["date", "holiday"])
        de_holidays_df['date'] = pd.to_datetime(de_holidays_df['date'])
        de_holidays_df.set_index('date', inplace=True)
        
        at_holiday_list = []
        for holiday in holidays.Austria(years=[2021,2022,2023]).items():
            at_holiday_list.append(holiday)
        at_holidays_df = pd.DataFrame(at_holiday_list, columns=["date", "holiday"])
        at_holidays_df['date'] = pd.to_datetime(at_holidays_df['date'])
        at_holidays_df.set_index('date', inplace=True)

        # add DE holidays to df
        merged_df = df.merge(de_holidays_df, how='left', left_index=True, right_index=True)
        merged_df['de_holiday'] = merged_df.index.isin(de_holidays_df.index).astype(int)
        merged_df.drop(columns=['holiday'], inplace=True)
        
        # add AT holidays to df
        at_holidays_df['at_holiday'] = 1 # add a 1 column to austrian holidays dataframe to help us merge with DE holidays
        merged_df = merged_df.merge(at_holidays_df[['at_holiday']], how='left', left_index=True, right_index=True)
        merged_df.head() # creates two columns (at_holiday_x, at_holiday_y), we only need one
        merged_df['at_holiday'].fillna(0, inplace=True) # replace NaN (no holiday) with 0
        merged_df['at_holiday'] = merged_df['at_holiday'].astype(int) # convert 1 and 0 to integers
        
        # combine columns
        merged_df['holiday'] = (merged_df['at_holiday'] | merged_df['de_holiday']).astype(int)
        merged_df = merged_df.drop(columns = ['de_holiday', 'at_holiday']) # drop individual DE and AT rows
        
        return merged_df[['holiday']]#.reset_index(drop=True)


### Weekends

In [6]:
class AddWeekendsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        """Creates a new column with row value = 1 if the day is a Friday or Saturday and 0 if not."""
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        weekday_values = df.index.weekday
        df['fri_sat'] = ((weekday_values == 4) | (weekday_values == 5)).astype(int)
        return df[['fri_sat']]

### Cyclical months

In [7]:
class AddMonthsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        df = (df
        .pipe(add_time_features, month=True)
        )
        
        months_in_a_year = 12

        df['sin_MonthYear'] = np.sin(2*np.pi*(df['month'])/months_in_a_year)
        df['cos_MonthYear'] = np.cos(2*np.pi*(df['month'])/months_in_a_year)
        df.drop(columns=['month'], inplace=True)
        
        return df[['sin_MonthYear', 'cos_MonthYear']]

In [8]:
gg_carryover_transformation = GeneralGaussianCarryover(tails = 'both')

## Adstock
* comes from `mamimo` package, explained [here](https://towardsdatascience.com/an-upgraded-marketing-mix-modeling-in-python-5ebb3bddc1b6)

In [9]:
adstock1 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', BoxCoxSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', BoxCoxSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', BoxCoxSaturation())
     ]), ['tiktok'])
])

adstock2 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['tiktok'])
])

adstock3 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', HillSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', HillSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', HillSaturation())
     ]), ['tiktok'])
])

adstock4 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['tiktok'])
])

adstock5 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', ExponentialSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', ExponentialSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', ExponentialSaturation())
     ]), ['tiktok'])
])

adstock6 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', BoxCoxSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', BoxCoxSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', gg_carryover_transformation),
            ('saturation', BoxCoxSaturation())
     ]), ['tiktok'])
])

adstock7 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', AdbudgSaturation())
     ]), ['tiktok'])
])

adstock8 = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', HillSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', HillSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', GeneralGaussianCarryover()),
            ('saturation', HillSaturation())
     ]), ['tiktok'])
])

## Time CT

In [10]:
time_features = ColumnTransformer(
    [
     ('holidays_pipe', Pipeline([
            ('add_holidays', AddHolidaysTransformer())
     ]), ['day']),
     ('weekends_pipe', Pipeline([
            ('add_weekends', AddWeekendsTransformer())
     ]), ['day']),
     ('months_pipe', Pipeline([
            ('add_months', AddMonthsTransformer())
     ]), ['day'])
], remainder = 'passthrough')

In [11]:
df = X_train.copy()
transformed_df = time_features.fit_transform(df)
transformed_df = pd.DataFrame(transformed_df, columns = ['holiday', 'weekends', 'sin', 'cos', 'facebook', 'google','tiktok'])
transformed_df

,holiday,weekends,sin,cos,facebook,google,tiktok
0,0.0,0.0,-0.500000,-0.866025,257.01,1.17,0.00
1,0.0,1.0,-0.500000,-0.866025,250.30,0.84,0.00
2,0.0,1.0,-0.500000,-0.866025,248.59,0.72,0.00
3,0.0,0.0,-0.500000,-0.866025,258.05,0.07,0.00
4,0.0,0.0,-0.500000,-0.866025,257.47,0.38,0.00
...,...,...,...,...,...,...,...
650,0.0,0.0,0.866025,-0.500000,885.33,208.52,209.03
651,0.0,0.0,0.866025,-0.500000,882.17,186.47,208.82
652,0.0,1.0,0.866025,-0.500000,662.08,160.69,216.38
653,0.0,1.0,0.866025,-0.500000,481.71,149.16,239.65


# Pipeline

In [12]:
model = LinearRegression()

adstock_pipeline1 = Pipeline([
    ('adstock1', adstock1)
])
adstock_pipeline2 = Pipeline([
    ('adstock2', adstock2)
])
adstock_pipeline3 = Pipeline([
    ('adstock3', adstock3)
])
adstock_pipeline4 = Pipeline([
    ('adstock4', adstock4)
])
adstock_pipeline5 = Pipeline([
    ('adstock5', adstock5)
])
adstock_pipeline6 = Pipeline([
    ('adstock6', adstock6)
])
adstock_pipeline7 = Pipeline([
    ('adstock7', adstock7)
])
adstock_pipeline8 = Pipeline([
    ('adstock8', adstock8)
])

In [13]:
# Define the number of adstock pipelines you want
num_pipelines = 8

# Create a list to store the adstock pipelines
pipelines = []

for i in range(1, num_pipelines + 1):
    # Create the adstock pipeline (as you've defined previously)
    adstock_pipeline = Pipeline([
        ('adstock{}'.format(i), globals()['adstock{}'.format(i)])
    ])
    
    time_preprocessing = Pipeline([
        ('time_features', time_features)
    ])
    
    # Combine the adstock and time_preprocessing pipelines into a FeatureUnion
    union = FeatureUnion([
        ('time_preprocessing', time_preprocessing),
        ('adstock_pipeline', adstock_pipeline)
    ])
    
    # Create the final pipeline
    pipeline = Pipeline([
        ('union', union),
        ('model', model)
    ])
    
    # Append the pipeline to the list
    pipelines.append(pipeline)


## model 1

In [14]:
param_grid_ = {
    'union__adstock_pipeline__adstock1__tiktok_pipe__saturation__shift': stats.randint(8,12),
    'union__adstock_pipeline__adstock1__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock1__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock1__tiktok_pipe__carryover__strength': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock1__google_pipe__saturation__shift': stats.randint(6,10),
    'union__adstock_pipeline__adstock1__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock1__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock1__google_pipe__carryover__strength': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock1__facebook_pipe__saturation__shift': stats.randint(5,9),
    'union__adstock_pipeline__adstock1__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock1__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock1__facebook_pipe__carryover__strength': [0.4,0.5,0.6]
}

param_grid = {'union__adstock_pipeline__adstock1__tiktok_pipe__saturation__shift': [10],
              'union__adstock_pipeline__adstock1__tiktok_pipe__saturation__exponent': [0.8],
              'union__adstock_pipeline__adstock1__tiktok_pipe__carryover__window': [3],
              'union__adstock_pipeline__adstock1__tiktok_pipe__carryover__strength': [0.5],
              'union__adstock_pipeline__adstock1__google_pipe__saturation__shift': [8],
              'union__adstock_pipeline__adstock1__google_pipe__saturation__exponent': [1.1],
              'union__adstock_pipeline__adstock1__google_pipe__carryover__window': [2],
              'union__adstock_pipeline__adstock1__google_pipe__carryover__strength': [1.0],
              'union__adstock_pipeline__adstock1__facebook_pipe__saturation__shift': [7],
              'union__adstock_pipeline__adstock1__facebook_pipe__saturation__exponent': [1.1],
              'union__adstock_pipeline__adstock1__facebook_pipe__carryover__window': [1],
              'union__adstock_pipeline__adstock1__facebook_pipe__carryover__strength': [0.5]}

In [15]:
grid_search = GridSearchCV(pipelines[0], param_grid, cv = tscv)
grid_search.fit(X_train, y_train)
best_pipeline = grid_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.32202566773983377'

In [16]:
random_search = RandomizedSearchCV(pipelines[0], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.30110877604355946'

In [17]:
## model 2

In [18]:
param_grid_ = {
    'union__adstock_pipeline__adstock2__tiktok_pipe__saturation__denominator_shift': stats.randint(8,12),
    'union__adstock_pipeline__adstock2__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock2__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock2__tiktok_pipe__carryover__strength': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock2__google_pipe__saturation__denominator_shift': stats.randint(6,10),
    'union__adstock_pipeline__adstock2__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock2__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock2__google_pipe__carryover__strength': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock2__facebook_pipe__saturation__denominator_shift': stats.randint(5,9),
    'union__adstock_pipeline__adstock2__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock2__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock2__facebook_pipe__carryover__strength': [0.4,0.5,0.6]}

In [19]:
random_search = RandomizedSearchCV(pipelines[1], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.3113302238185609'

In [20]:
## model 3

In [21]:
param_grid_ = {
    'union__adstock_pipeline__adstock3__tiktok_pipe__saturation__half_saturation': stats.randint(8,12),
    'union__adstock_pipeline__adstock3__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock3__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock3__tiktok_pipe__carryover__strength': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock3__google_pipe__saturation__half_saturation': stats.randint(6,10),
    'union__adstock_pipeline__adstock3__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock3__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock3__google_pipe__carryover__strength': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock3__facebook_pipe__saturation__half_saturation': stats.randint(5,9),
    'union__adstock_pipeline__adstock3__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock3__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock3__facebook_pipe__carryover__strength': [0.4,0.5,0.6]}

In [22]:
random_search = RandomizedSearchCV(pipelines[2], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.3224085503307459'

In [23]:
## model 4

In [24]:
param_grid_ = {
    'union__adstock_pipeline__adstock4__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock4__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock4__tiktok_pipe__carryover__strength': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock4__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock4__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock4__google_pipe__carryover__strength': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock4__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock4__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock4__facebook_pipe__carryover__strength': [0.4,0.5,0.6]}

In [25]:
random_search = RandomizedSearchCV(pipelines[3], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.3334620593376765'

In [26]:
## model 5

In [27]:
param_grid_ = {
    'union__adstock_pipeline__adstock5__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock5__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock5__tiktok_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock5__tiktok_pipe__carryover__sig': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock5__tiktok_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock5__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock5__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock5__google_pipe__carryover__p': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock5__google_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock5__google_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock5__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock5__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock5__facebook_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock5__facebook_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock5__facebook_pipe__carryover__tails': ['left', 'right', 'both'],}

In [28]:
random_search = RandomizedSearchCV(pipelines[4], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.33643524257586555'

In [29]:
## model 6

In [30]:
param_grid_ = {
    'union__adstock_pipeline__adstock6__tiktok_pipe__saturation__shift': stats.randint(8,12),
    'union__adstock_pipeline__adstock6__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock6__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock6__tiktok_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock6__tiktok_pipe__carryover__sig': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock6__tiktok_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock6__google_pipe__saturation__shift': stats.randint(6,10),    
    'union__adstock_pipeline__adstock6__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock6__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock6__google_pipe__carryover__p': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock6__google_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock6__google_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock6__facebook_pipe__saturation__shift': stats.randint(5,9),
    'union__adstock_pipeline__adstock6__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock6__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock6__facebook_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock6__facebook_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock6__facebook_pipe__carryover__tails': ['left', 'right', 'both'],}

In [31]:
random_search = RandomizedSearchCV(pipelines[5], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.2731874521476767'

In [32]:
## model 7

In [33]:
param_grid_ = {
    'union__adstock_pipeline__adstock7__tiktok_pipe__saturation__denominator_shift': stats.randint(8,12),
    'union__adstock_pipeline__adstock7__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock7__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock7__tiktok_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock7__tiktok_pipe__carryover__sig': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock7__tiktok_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock7__google_pipe__saturation__denominator_shift': stats.randint(6,10),    
    'union__adstock_pipeline__adstock7__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock7__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock7__google_pipe__carryover__p': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock7__google_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock7__google_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock7__facebook_pipe__saturation__denominator_shift': stats.randint(5,9),
    'union__adstock_pipeline__adstock7__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock7__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock7__facebook_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock7__facebook_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock7__facebook_pipe__carryover__tails': ['left', 'right', 'both']}

In [34]:
random_search = RandomizedSearchCV(pipelines[6], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.30438966389199573'

In [35]:
## model 8

In [36]:
param_grid_ = {
    'union__adstock_pipeline__adstock8__tiktok_pipe__saturation__exponent': [0.7,0.8,0.9],
    'union__adstock_pipeline__adstock8__tiktok_pipe__saturation__half_saturation': stats.randint(8,12),
    'union__adstock_pipeline__adstock8__tiktok_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock8__tiktok_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock8__tiktok_pipe__carryover__sig': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock8__tiktok_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock8__google_pipe__saturation__exponent': [0.9,1.1,1.2],
    'union__adstock_pipeline__adstock8__google_pipe__saturation__half_saturation': stats.randint(8,12),
    'union__adstock_pipeline__adstock8__google_pipe__carryover__window': stats.randint(1,5),
    'union__adstock_pipeline__adstock8__google_pipe__carryover__p': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock8__google_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock8__google_pipe__carryover__tails': ['left', 'right', 'both'],
    'union__adstock_pipeline__adstock8__facebook_pipe__saturation__exponent': [1.0,1.1,1.2],
    'union__adstock_pipeline__adstock8__facebook_pipe__saturation__half_saturation': stats.randint(8,12),
    'union__adstock_pipeline__adstock8__facebook_pipe__carryover__window': stats.randint(1,3),
    'union__adstock_pipeline__adstock8__facebook_pipe__carryover__p': [0.4,0.5,0.6],
    'union__adstock_pipeline__adstock8__facebook_pipe__carryover__sig': [0.9,1.0,1.1],
    'union__adstock_pipeline__adstock8__facebook_pipe__carryover__tails': ['left', 'right', 'both'],}

In [37]:
random_search = RandomizedSearchCV(pipelines[7], param_grid_, n_iter=500, n_jobs = -1, cv = tscv)
random_search.fit(X_train, y_train)
best_pipeline = random_search.best_estimator_

f"R-squared: {best_pipeline.score(X_test, y_test)}"

'R-squared: 0.3164999465450635'